<a href="https://colab.research.google.com/github/inigmat/exupery/blob/main/SandDistribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Simple transportation problem (distributing materials across the sites)**

Five construction sites utilize sand extracted from three quarries. The daily productivity of each quarry (column 'Stock pile'), the sand requirement for each site, and the distances from quarries to consumers are presented in the table.

***Distances from the stockpiles to the sites and requested quantity of sand***

| Stock pile | Site 1 | Site 2 | Site 3 | Site 4 | Site 5 |
| --- | --- | --- | --- | --- | --- |
| 500 | 2 | 4 | 3 | 8 | 1 |
| 400 | 5 | 6 | 4 | 7 | 2 |
| 900 | 1 | 3 | 5 | 4 | 7 |
| Requested Qty, tonnes | 300 | 200 | 400 | 600 | 300 |

Solve the problem of a optimal sand transportation plan that minimizes the total transportation in ton-kilometers.

## **Installing CPLEX and docplex if they are not installed and reloading runtime**


In [1]:
import sys

try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        !pip install docplex
    else:
        !pip install --user docplex

try:
    import cplex
except:
    if hasattr(sys, 'real_prefix'):
        !pip install cplex
    else:
        !pip install --user cplex
        exit()

## **Solve the problem**

In [2]:
from docplex.mp.model import Model

# Data from the table
stock_piles = [500, 400, 900]
sites = [1, 2, 3, 4, 5]
requested_qty = [300, 200, 400, 600, 300]
distances = [
    [2, 4, 3, 8, 1],
    [5, 6, 4, 7, 2],
    [1, 3, 5, 4, 7]
]

# Enumerate the data for using it in code
stock_piles_nums = range(len(stock_piles))
sites_nums = range(len(sites))

# Create a model
model = Model(name='sand_transportation')

# Decision variables - amount of sand transported from each stockpile to each site
x = {(i, j): model.continuous_var(name='x_{0}_{1}'.format(i, j)) for i in stock_piles_nums for j in sites_nums}

# Constraints on the productivity of stockpiles
for i in stock_piles_nums:
    model.add_constraint(model.sum(x[i, j] for j in sites_nums) <= stock_piles[i])

# Constraints on the sand consumption at sites
for j in sites_nums:
    model.add_constraint(model.sum(x[i, j] for i in stock_piles_nums) == requested_qty[j])

# Minimize the total transportation distance
model.minimize(model.sum(distances[i][j] * x[i, j] for i in stock_piles_nums for j in sites_nums))

# Solve the problem
solution = model.solve()

# Output the results
print(solution)

solution for: sand_transportation
objective: 5400
status: OPTIMAL_SOLUTION(2)
x_0_0=200.000
x_0_4=300.000
x_1_2=400.000
x_2_0=100.000
x_2_1=200.000
x_2_3=600.000



## **Get the results in table**

In [3]:
import pandas as pd

# Create a Dataframe for the results
distribution_df = pd.DataFrame(columns=[f'Site {j}' for j in sites_nums],
                         index=[f'Stockpile {i}' for i in stock_piles_nums])

# Fill the DataFrame with values from the solution
for i in stock_piles_nums:
    for j in sites_nums:
        distribution_df.at[f'Stockpile {i}', f'Site {j}'] = x[i, j].solution_value

# Display the DataFrame
distribution_df

,Site 0,Site 1,Site 2,Site 3,Site 4
Stockpile 0,200.0,0,0,0,300.0
Stockpile 1,0,0,400.0,0,0
Stockpile 2,100.0,200.0,0,600.0,0


# The results can be presented in a diagram

Blank document.svg

> Блок с отступами

